In [1]:
from platform import python_version

print(python_version())

import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem

from mendeleev import element

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#import chemcoord as cc

from rdkit import Chem
import seaborn as sns
import pandas as pd
import numpy as np
import shutil
import glob
import csv
import ast
import os
import re
#import rmsd

sns.set_context('poster', font_scale=1.1)
pd.set_option('display.max_rows', None)  

3.10.6


In [2]:
# set folder location
comp_loc = '/Users/z5380625/Documents/Research/'
folder_loc = '17_Atoms'   

# set range for atoms
data = pd.read_csv(comp_loc+'4_BigData/'+folder_loc+'/biosig_CHNOPS.csv')
data = data.query("CHNOPS == 'Y'")
df_molecules = data.loc[(data['Tot_atoms'] == 17)]
df_molecules

,Unnamed: 0,Formula,Formula_ID,Tot_atoms,Tot_nH_atoms,Tot_H_atoms,SMILES,Dataset_no,Molecular_weight,IUPAC_chemname,...,EPISUITE,InChI_Code,InChI_Key,Boiling_Point,BP_basis,EPIsuite_output,MP,Basis,Life,CHNOPS
565,565,C5H12,C5H12_1,17,5,12,CC(CC)C,662,72.151,isopentane,...,CC(CC)C,"InChI=1S/C5H12/c1-4-5(2)3/h5H,4H2,1-3H3",InChIKey=QWTDNUCVQCZILF-UHFFFAOYSA-N,27.55462649,Ex,30.18(BP est) -119.04(MP est) 27.80(...,-159.9,Expt,Y,Y
566,566,C5H12,C5H12_2,17,5,12,CC(C)(C)C,663,72.151,neopentane,...,CC(C)(C)C,"InChI=1S/C5H12/c1-5(2,3)4/h1-4H3",InChIKey=CRSOQBOWXPBRES-UHFFFAOYSA-N,9.205,Ex,19.60(BP est) -113.42(MP est) 9.50(...,-16.4,Expt,N,Y
567,567,C5H12,C5H12_3,17,5,12,CCCCC,664,72.151,pentane,...,CCCCC,"InChI=1S/C5H12/c1-3-5-4-2/h3-5H2,1-2H3",InChIKey=OFBQJSOFQDEBGM-UHFFFAOYSA-N,36.02657459,Ex,46.01(BP est) -106.92(MP est) 36.00(...,-129.7,Expt,Y,Y
1644,1644,C5H11N,C5H11N_1,17,6,11,NCC1(CC1)C,1850,85.150,(1-methylcyclopropyl)methanamine,...,NCC1(CC1)C,"InChI=1S/C5H11N/c1-5(4-6)2-3-5/h2-4,6H2,1H3",InChIKey=NIJMHZCHTZXLJX-UHFFFAOYSA-N,95.55,Ex,103.22(BP est) -27.67(MP est) ----(...,-27.67,Pred,N,Y
1645,1645,C5H11N,C5H11N_2,17,6,11,NCC1C(C)C1,1851,85.150,(2-methylcyclopropyl)methanamine,...,NCC1C(C)C1,"InChI=1S/C5H11N/c1-4-2-5(4)3-6/h4-5H,2-3,6H2,1H3",InChIKey=BRWRRWXQMCPPRZ-UHFFFAOYSA-N,108.88,Pr,108.88(BP est) -40.09(MP est) ----(...,-40.09,Pred,N,Y
1646,1646,C5H11N,C5H11N_3,17,6,11,NC(C1CC1)C,1852,85.150,1-cyclopropylethan-1-amine,...,NC(C1CC1)C,"InChI=1S/C5H11N/c1-4(6)5-2-3-5/h4-5H,2-3,6H2,1H3",InChIKey=IXCXVGWKYIDNOS-UHFFFAOYSA-N,101.42,Pr,101.42(BP est) -47.65(MP est) ----(...,-47.65,Pred,N,Y
1647,1647,C5H11N,C5H11N_4,17,6,11,CNCC1CC1,1853,85.150,1-cyclopropyl-N-methylmethanamine,...,CNCC1CC1,"InChI=1S/C5H11N/c1-6-4-5-2-3-5/h5-6H,2-4H2,1H3",InChIKey=BFPMCZWKUSUMKE-UHFFFAOYSA-N,95,Ex,97.11(BP est) -56.70(MP est) ----(...,-56.7,Pred,N,Y
1648,1648,C5H11N,C5H11N_5,17,6,11,NC1(CC)CC1,1854,85.150,1-ethylcyclopropan-1-amine,...,NC1(CC)CC1,"InChI=1S/C5H11N/c1-2-5(6)3-4-5/h2-4,6H2,1H3",InChIKey=RTHXBEOKSSCNNT-UHFFFAOYSA-N,103.22,Pr,103.22(BP est) -27.67(MP est) ----(...,-27.67,Pred,N,Y
1649,1649,C5H11N,C5H11N_6,17,6,11,NC1(CCC1)C,1855,85.150,1-methylcyclobutan-1-amine,...,NC1(CCC1)C,"InChI=1S/C5H11N/c1-5(6)3-2-4-5/h2-4,6H2,1H3",InChIKey=ZAXBVBGWLMVNJN-UHFFFAOYSA-N,84.38,Ex,105.41(BP est) -28.79(MP est) ----(...,-28.79,Pred,N,Y
1650,1650,C5H11N,C5H11N_7,17,6,11,CN1CCCC1,1856,85.150,1-methylpyrrolidine,...,CN1CCCC1,"InChI=1S/C5H11N/c1-6-4-2-3-5-6/h2-5H2,1H3",InChIKey=AVFZOVWCLRSYKC-UHFFFAOYSA-N,79.385,Ex,91.60(BP est) -49.97(MP est) 81.00(...,-49.97,Pred,Y,Y


In [71]:
mol = data.loc[data['IUPAC_chemname'] == 'methyl formate'] #C#CCC#N
mol

,Unnamed: 0,Formula,Formula_ID,Tot_atoms,Tot_nH_atoms,Tot_H_atoms,SMILES,Dataset_no,Molecular_weight,IUPAC_chemname,...,EPISUITE,InChI_Code,InChI_Key,Boiling_Point,BP_basis,EPIsuite_output,MP,Basis,Life,CHNOPS
175,175,C2H4O2,C2H4O2_4,8,4,4,O=COC,213,60.052,methyl formate,...,O=COC,"InChI=1S/C2H4O2/c1-4-2-3/h2H,1H3",InChIKey=TZIHFWKZFHZASV-UHFFFAOYSA-N,31.41378472,Ex,28.88(BP est) -99.48(MP est) 31.70(...,-99,Expt,Y,Y


# 0. Generate geometries from SMILES identifiers

In [60]:
for smiles, formula in zip(df_molecules['SMILES'], df_molecules['Formula_ID']):

    try:
        mol = Chem.MolFromSmiles(smiles)
        molh = Chem.AddHs(mol)
        AllChem.EmbedMolecule(molh)
        AllChem.MMFFOptimizeMolecule(molh)

        position_list = []
        for atom in molh.GetConformers():
            position_list.append(atom.GetPositions())

        # Stores atomic symbols
        atoms_list = []
        for atom in molh.GetAtoms():
            atoms_list.append(atom.GetSymbol())

        # Merges both the molecular coordinates and atomic symbols into a readable format
        final_array = np.column_stack((atoms_list, position_list[0]))

#         with open(
#                 comp_loc+'2_BigData/' + folder_loc +
#                 '/0_GeomsFromSmiles/' + formula + '.xyz', 'a') as inp:
#             inp.write(str('atom' + '\t\t x' + '\t\t y' + '\t\t z\n\n'))
#             for row in final_array:
#                 print('\t'.join(map(str, row)), file=inp)

    except:

        print(smiles, formula)

[11:16:59] UFFTYPER: Warning: hybridization set to SP3 for atom 1


# 1. Geometry optimisation using xtb

## 1.1 Input files for initial geometry optimisation

In [82]:
# The first step for the xtb opt is to rewrite the input files in a format readable by xtb

QC_loc = comp_loc+'4_BigData/'+folder_loc+'/0_GeomsFromSmiles/'


from mendeleev import element

# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)
files_names = glob.glob(QC_loc+'*.xyz')
failed_jobs = []

for file in files_names:
    
    filen = file.split('/')[-1].split('.xyz')[0]
    
    os.mkdir(comp_loc+'4_BigData/'+folder_loc+'/1_xtb_Optimisation/InputFiles/'+filen)
    
    geom_list = []
    with open(QC_loc+filen+'.xyz', 'r') as inp:
        for line in inp:
            if line.startswith('atom'):
                continue
            elif line.startswith('\n'):
                continue
            else:
                geom_list.append(line)
        num_atoms = len(geom_list)
        with open(comp_loc+'4_BigData/'+folder_loc+'/1_xtb_Optimisation/InputFiles/'+filen+'/'+filen+'.xyz', 'a') as inp:
                    inp.write(str(int(num_atoms))+'\n\n')
                    for line in geom_list:
                        print(line.rstrip(), file = inp)
                    inp.write('\n')
                
        
        

In [83]:
# --------------------------------------------------------------
# Input File Generation for Gaussian Crude Geometry Calculation
# --------------------------------------------------------------

#wB97XD
temp_loc = comp_loc+'4_BigData/'+folder_loc+'/Templates/'
QC_loc = comp_loc+'4_BigData/'+folder_loc+'/1_xtb_Optimisation/InputFiles/'

from mendeleev import element

# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)
files_names = glob.glob(comp_loc+'4_BigData/'+folder_loc+'/0_GeomsFromSmiles/*.xyz')
two_mult = ['CH','HCO','NO2','OH','SH','BO','ClO','CN','HO2']

molecules_considered = list(df_molecules['Formula_ID'])

for file in files_names:
    
    
    filen = file.split('/')[-1].split('.xyz')[0]
    
    if filen in molecules_considered:
        try:
            # Copies submission file template and pastes the appropriate input file name.
            shutil.copy(temp_loc+'geomopt_xtb_submission.pbs',QC_loc+filen+'/'+filen+'_xtb.pbs')
            with open(QC_loc+filen+'/'+filen+'_xtb.pbs','r+') as inp_sub:
                submission_data = inp_sub.read()
                submission_data = re.sub('{molecule}', filen.rstrip(), submission_data)
                submission_data = re.sub('{time}', '01:00:00', submission_data)
                submission_data = re.sub('{mem}', '32', submission_data)
                submission_data = re.sub('{cpus}', '8', submission_data)

                inp_sub.seek(0)
                inp_sub.write(submission_data)
                inp_sub.truncate()
                
        except:
            
            print(filen)
        

## 1.2 Parsing final geometry from output files

In [49]:
# The first step for the CENSO run is to extract the converged geometry from the Gaussian output file
# to set it as an input for CREST. This will print all conformers avaibale for that geometry.

QC_loc = comp_loc+'4_BigData/'+folder_loc+'/1_QC_Optimisation/'

from mendeleev import element

# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)
files_names = glob.glob(QC_loc+'OutputFiles/*.log')
failed_jobs = []

for file in files_names:
    
    filen = file.split('/')[-1].split('_qcopt')[0]

    with open(QC_loc+'OutputFiles/'+filen+'_qcopt.log', 'r') as inp:
        # Checks if the output file terminated noramlly.
        chk_count = 0
        for g in inp:
            if 'Normal termination' in g:
                chk_count = chk_count + 1

        # If output file is okay, proceeds with extracting the geometires.
        if chk_count != 0:
            
            try:
                
                # Copies all geometries from output file and stores them into a list (all_geoms).
                with open(QC_loc+'OutputFiles/'+filen+'_qcopt.log', 'r') as inp:
                    
                    all_geoms = []
                    copy = False
                    for line in inp:
                        line = line.strip()
                        if 'Input orientation:' in line:
                            copy = True
                        if copy:
                            all_geoms.append(line)
                        strings = ('Symmetry turned off by external request','Distance matrix')
                        if any(s in line for s in strings):
                            copy = False

                    # From all geometries, stores the final geometry into the up_geom list 
                    # (this corresponds to the converged geometrey).
                    up_geom = []
                    count = 0
                    for line in reversed(all_geoms):
                        if count < 1:
                            if 'Input orientation:' not in line:
                                up_geom.append(line)
                            else:
                                count = count + 1
                
                    # Removes unnecesary words from converged geometry.
                    up_geom = list(filter(lambda line: not line.startswith('-'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('C'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('R'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('N'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('D'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('S'), up_geom))
                    
                    # Replaces atomic numbers with atomic symbols.
                    final_geom = []
                    for line in up_geom:
                        values_final_geom = line.split()
                        atom_symbol = element(int(values_final_geom[1]))
                        values_final_geom[1] = str(atom_symbol.symbol)
                        final_geom.append(' '.join(values_final_geom))
                    
                    # Copies the final geometry into the corresponding file with the total number of atoms.
                    with open(QC_loc+'CREST_IntGeoms/'+filen+'_intgeom.xyz','a') as f:
                        f.write(str(len(final_geom))+'\n\n')
                        for value in reversed(final_geom):
                            values = value.split()
                            print(values[1]+" "+values[3]+" "+values[4]+" "+values[5], file = f)
                        
            except:
                failed_jobs.append(filen)
        
        # If output file has problems, prints the name of the file.
        else:
            failed_jobs.append(filen)
            print(filen)

# 2. CREST and CENSO

## 2.1 Generating input files for both CREST and CENSO

In [99]:
# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)

QC_loc = comp_loc+'4_BigData/'+folder_loc+'/1_xtb_Optimisation/'
CRENSO_loc = comp_loc+'4_BigData/'+folder_loc+'/2_CREST_CENSO_Inputs/'
temp_loc = comp_loc+'4_BigData/'+folder_loc+'/Templates/'

files_names = glob.glob(QC_loc+'xtb_Geoms/*.xyz')
failed_jobs = []

for file in files_names:
    
    filen = file.split('/')[-1].split('.xyz')[0]
    
    # Creates individual folders for each molecule
    os.mkdir(CRENSO_loc+filen)
    
    # Copies final geometry from QC calculation into corresponding molecular folder
    shutil.copy(QC_loc+'xtb_Geoms/'+filen+'.xyz', CRENSO_loc+filen+'/'+filen+'_intgeom.xyz')
    
    # Creates submission file for running CREST for each molecule indivudally (inside corresponding folder)
    shutil.copy(temp_loc+'CREST_template.pbs', CRENSO_loc+filen+'/'+filen.rstrip()+'_CREST.pbs')
    with open(CRENSO_loc+filen+'/'+filen.rstrip()+'_CREST.pbs', 'r+') as inp_sub:
        submission_data = inp_sub.read()
        submission_data = re.sub('{molecule}', filen.rstrip(), submission_data)

        inp_sub.seek(0)
        inp_sub.write(submission_data)
        inp_sub.truncate()

    # Creates submission file for running CENSO for CONFORMERS
    shutil.copy(temp_loc+'CENSO_template_conformers.pbs', CRENSO_loc+filen+'/'+filen.rstrip()+'_CENSO_conformers.pbs')
    with open(CRENSO_loc+filen+'/'+filen.rstrip()+'_CENSO_conformers.pbs', 'r+') as inp_sub:
        submission_data = inp_sub.read()
        submission_data = re.sub('{molecule}', filen.rstrip(), submission_data)

        inp_sub.seek(0)
        inp_sub.write(submission_data)
        inp_sub.truncate()
        
#     # Creates submission file for running CENSO for ROTAMERS
#     shutil.copy(temp_loc+'CENSO_template_rotamers.pbs', CRENSO_loc+filen+'/'+filen.rstrip()+'_CENSO_rotamers.pbs')
#     with open(CRENSO_loc+filen+'/'+filen.rstrip()+'_CENSO_rotamers.pbs', 'r+') as inp_sub:
#         submission_data = inp_sub.read()
#         submission_data = re.sub('{molecule}', filen.rstrip(), submission_data)

#         inp_sub.seek(0)
#         inp_sub.write(submission_data)
#         inp_sub.truncate()
        
        

# 3. Sorting and organising final geometries

In [20]:
## Change Conformers/Rotamer Extensions ##
extension = ['Conformers','conformers','CONF'] #['Rotamers','rotamers','ROTA'] #

files_names = glob.glob(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/BWeights_GeomFiles_'+extension[0]+'/*.xyz')
failed_jobs = []

for file in files_names:
    
    filen = file.split('/')[-1].split('_fconfs')[0]
    
    try:

        with open(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/BWeights_GeomFiles_'+extension[0]+'/'+filen+'_fconfs_'+extension[1]+'.xyz') as inp:

            # Stores all conformers geometries in the all_geometries list and the conformer number in conf_number.
            # Keeps track of the number of lines for each conformer:
            # ' total atoms' line + 'G(CENSO)' line + lines for coordinates
            all_geometries = []
            conf_number = []
            
            for line in inp:
                all_geometries.append(line)
                if line.startswith(' '):
                    tot_lines = int(line.rstrip().split(' ')[-1]) + 2
                if line.startswith('G'):
                    conf_number.append(line.rstrip().split('!')[-1].split('CONF')[-1])

            # Splits all geometries into sublists for each individual conformer.abs
            conf_geometries = []
            for line in range(0, len(all_geometries), tot_lines):
                conf_geometries.append(all_geometries[line:line+tot_lines])

            # Pastes conformers geometries in individual files.
            for cnumber,conformer in zip(conf_number,conf_geometries):
                # Removes unnecessary lines from lists.
                conformer = list(filter(lambda line: not line.startswith(' '), conformer))
                conformer = list(filter(lambda line: not line.startswith('G'), conformer))
                with open(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/FinalGeoms_'+extension[0]+'/'+filen+'_'+extension[2]+cnumber+'.xyz', 'a') as inp:
                    inp.write(str('atom'+'\t x'+'\t\t y'+'\t\t z\n\n'))
                    for line in conformer:
                        print(line.rstrip(), file = inp)
                        
    except:
        print(filen)
        
        

# 4. Vibrational frequency calculations

## 4.1 Input files for vibrational calculations

In [21]:
## Change Conformers/Rotamer Extensions ##

extension = ['Conformers','conformers','CONF'] #

# ----------------------------------------------------------
# Input File Generation for Gaussian Vibrational Calculation
# ----------------------------------------------------------

from mendeleev import element

# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)
files_names = glob.glob(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/FinalGeoms_'+extension[0]+'/*.xyz')
two_mult = ['CH','HCO','NO2','OH','SH','BO','ClO','CN','HO2']

molecules_considered = list(df_molecules['Formula_ID'])

for file in files_names:

    filen = file.split('/')[-1].split('.xyz')[0].split('_'+extension[2])[0]
    filen = filen.split('p')[0]
    filename = file.split('/')[-1].split('.xyz')[0]
    if filen in molecules_considered:
        try:
            # Copies the template file, changing its name to the one for the molecule being considered.
            # https://thispointer.com/python-how-to-copy-files-from-one-location-to-another-using-shutil-copy/
            shutil.copy(comp_loc+'4_BigData/'+folder_loc+'/Templates/harm_gaussian.gjf', comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_'+extension[0]+'/'+filename+'_harmonic_'+extension[1]+'.gjf')
            
            basis_data = []
            elements_list = []
            atomic_numbers = []
            basis_str = ''

            # These for loops store the element composition in the molecular formula including the atomic numbers.
            for letter in re.findall('[A-Z][^A-Z]*', filen.rstrip().split('_'+extension[2])[0]):
                symbol = re.split(r'[0-9]', letter.split()[0])[0]
                poss_under = re.split(r'(_)', symbol.split()[0])[0]
                elements_list.append(poss_under)

            for atom in elements_list:
                        symbol = element(atom)
                        atomic_numbers.append(symbol.atomic_number)

            atomic_numbers.sort()

            # With the atomic number from the molecular formula, this part of the code stores the corresponding elements
            #from the user-defined basis set.
            basis_file = open(comp_loc+'4_BigData/'+folder_loc+'/UserBasisSet/Def2TZVPD.txt',"r")
            contents = basis_file.read()
            basis_dict = ast.literal_eval(contents)
            for number in atomic_numbers:
                if number in basis_dict:
                    basis_data.append(basis_dict[number])

            basis_str = '\n'.join(basis_data)

#             # Saves the info for the respective geometry into the geom_data list.
            with open(file,'r') as inp_geom:
                geom_data = inp_geom.read()
                geom_data = geom_data.split('\n',2)[-1]
                inp_geom.close()
                
            # Replaces the selected keywords (i.e. {molecule}, {geometry} and {basis}) with the
            # corresponding values.
            # https://www.kite.com/python/answers/how-to-update-and-replace-text-in-a-file-in-python
            with open(comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_'+extension[0]+'/'+filename+'_harmonic_'+extension[1]+'.gjf','r+') as inp_template:
                template_data = inp_template.read()
                template_data = re.sub('{cpu}', '8', template_data)
                template_data = re.sub('{mem}', '32', template_data)
                template_data = re.sub('{geometry}', geom_data, template_data)
                template_data = re.sub('{molecule}', filename.rstrip(), template_data)
                template_data = re.sub('{charge}', '0', template_data)
                template_data = re.sub('{basis_data}', basis_str, template_data)
                
                # Considering different values of multiplicity
                if filen in two_mult:
                    template_data = re.sub('{multiplicity}', '2', template_data)
                else:
                    template_data = re.sub('{multiplicity}', '1', template_data)
                
                inp_template.seek(0)
                inp_template.write(template_data)
                inp_template.truncate()
                
            # Copies submission file template and pastes the appropriate input file name.
            shutil.copy(comp_loc+'4_BigData/'+folder_loc+'/Templates/harm_gaussian_submission.pbs', comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_'+extension[0]+'/'+filename+'_harmonic_'+extension[1]+'.pbs')
            with open(comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/InputFiles_'+extension[0]+'/'+filename+'_harmonic_'+extension[1]+'.pbs','r+') as inp_sub:
                submission_data = inp_sub.read()
                submission_data = re.sub('{molecule}', filename.rstrip(), submission_data)
                submission_data = re.sub('{extension}', extension[1].rstrip(), submission_data)
                submission_data = re.sub('{time}', '04:00:00', submission_data)
                submission_data = re.sub('{mem}', '32', submission_data)
                submission_data = re.sub('{cpus}', '8', submission_data)
                submission_data = re.sub('{extension}', extension[1], submission_data)

                inp_sub.seek(0)
                inp_sub.write(submission_data)
                inp_sub.truncate()
                
        except:
            
            print(filen)
        

## 4.2 Get Final Geoms from VibCalc

In [3]:
# The first step for the CENSO run is to extract the converged geometry from the Gaussian output file
# to set it as an input for CREST. This will print all conformers avaibale for that geometry.
folder_loc = '17_Atoms'
extension = ['Conformers','conformers','CONF'] #['Rotamers','rotamers','ROTA'] #
vibcalc_loc = comp_loc+'4_BigData/'+folder_loc+'/4_VibrationalCalcs/'

from mendeleev import element

# Reads file names from the storage folder (CHANGE ACCORDING WITH THE MOLECULAR GROUP!!)
files_names = glob.glob(vibcalc_loc+'OutputFiles/*.log')
failed_jobs = []

for file in files_names:
    filen = file.split('/')[-1].split('_harmonic_')[0]

    with open(vibcalc_loc+'OutputFiles/'+filen+'_harmonic_'+extension[1]+'.log', 'r') as inp:
        # Checks if the output file terminated noramlly.
        chk_count = 0
        for g in inp:
            if 'Normal termination' in g:
                chk_count = chk_count + 1

        # If output file is okay, proceeds with extracting the geometires.
        if chk_count != 0:
            
            try:
                
                # Copies all geometries from output file and stores them into a list (all_geoms).
                with open(vibcalc_loc+'OutputFiles/'+filen+'_harmonic_'+extension[1]+'.log', 'r') as inp:
                    
                    all_geoms = []
                    copy = False
                    for line in inp:
                        line = line.strip()
                        if 'Input orientation:' in line:
                            copy = True
                        if copy:
                            all_geoms.append(line)
                        strings = ('Symmetry turned off by external request','Distance matrix')
                        if any(s in line for s in strings):
                            copy = False

                    # From all geometries, stores the final geometry into the up_geom list 
                    # (this corresponds to the converged geometrey).
                    up_geom = []
                    count = 0
                    for line in reversed(all_geoms):
                        if count < 1:
                            if 'Input orientation:' not in line:
                                up_geom.append(line)
                            else:
                                count = count + 1
                
                    # Removes unnecesary words from converged geometry.
                    up_geom = list(filter(lambda line: not line.startswith('-'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('C'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('R'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('N'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('D'), up_geom))
                    up_geom = list(filter(lambda line: not line.startswith('S'), up_geom))
                    
                    # Replaces atomic numbers with atomic symbols.
                    final_geom = []
                    for line in up_geom:
                        values_final_geom = line.split()
                        atom_symbol = element(int(values_final_geom[1]))
                        values_final_geom[1] = str(atom_symbol.symbol)
                        final_geom.append(' '.join(values_final_geom))
                    
                    # Copies the final geometry into the corresponding file with the total number of atoms.
                    with open(vibcalc_loc+'FinalGeoms_VibCalc/'+filen+'_VibCalcGeom.xyz','a') as f:
                        f.write(str(len(final_geom))+'\n\n')
                        for value in reversed(final_geom):
                            values = value.split()
                            print(values[1]+" "+values[3]+" "+values[4]+" "+values[5], file = f)
                        
            except:
                failed_jobs.append(filen)
        
        # If output file has problems, prints the name of the file.
        else:
            failed_jobs.append(filen)
            print(filen)

## 4.3 Get CREST Geometries

In [59]:
# # set folder location
# comp_loc = '/Users/z5380625/Documents/Research/'
folder_loc = '18_Atoms'  

# ## Change Conformers Extensions ##
# extension = ['Conformers','conformers','CONF'] 

#os.mkdir(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/FinalGeoms_CREST/')

files_names = glob.glob(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/CREST_Geoms/*.xyz')
failed_jobs = []

for file in files_names:
    
    filen = file.split('/')[-1]
    print(filen)
    try:

        with open(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/CREST_Geoms/'+filen) as inp:

            # Stores all conformers geometries in the all_geometries list and the conformer number in conf_number.
            # Keeps track of the number of lines for each conformer:
            # ' total atoms' line + 'G(CENSO)' line + lines for coordinates
            all_geometries = []
            conf_number = []
            x = inp.tell()
            firstLine = str(int(inp.readline().rstrip()))
            inp.seek(x)

            string = '  '+firstLine+'\n'
            n=0
            for line in inp:
                all_geometries.append(line)
                if string in line:
                    n+=1

            # Splits all geometries into sublists for each individual conformer.abs
            filen = filen.split('.xyz')[0]
            m = int((len(all_geometries)-n*2)/n+2)
            conf_geoms = [all_geometries[x:x+m] for x in range(0, len(all_geometries),m)]
            print(conf_geoms)
            cnumber = 0
            for conformer in conf_geoms:
                conformer = list(filter(lambda line: not line.startswith(string), conformer))
                conformer = list(filter(lambda line: not line.startswith('        '), conformer))
                print(conformer)

                with open(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/FinalGeoms_CREST/'+filen+'_'+str(cnumber)+'.xyz', 'a') as inp:
                    inp.write(str('atom'+'\t x'+'\t\t y'+'\t\t z\n\n'))
                    for line in conformer:
                        print(line.rstrip(), file = inp)
                cnumber+=1
                
                        
    except:
        print(filen, ' 2 atoms')
        filen = filen.split('.xyz')[0]
        conformer = all_geometries[2:]
        #print(conformer)
        cnumber = 0
        print(filen)
        with open(comp_loc+'4_BigData/'+folder_loc+'/3_CREST_CENSO_Outputs/FinalGeoms_CREST/'+filen+'_'+str(cnumber)+'.xyz', 'a') as inp:
            inp.write(str('atom'+'\t x'+'\t\t y'+'\t\t z\n\n'))
            for line in conformer:
                print(line.rstrip(), file = inp)
                    
        
            
        
        

C6H12_14_conformers.xyz
[['  18\n', '        -18.93711374\n', ' C         -0.5753748590        1.9432066047        0.0663697121\n', ' C         -0.0976742819        0.5058360714        0.2883379181\n', ' C         -1.0124432861       -0.4415302186       -0.4270891325\n', ' C         -1.7416165855       -1.3720441040        0.1584813008\n', ' C          1.3436374907        0.3395711148       -0.2119282012\n', ' C          1.9065375037       -1.0401249167        0.1056216775\n', ' H          0.0624101723        2.6436753688        0.6011211463\n', ' H         -0.5489734394        2.1928825851       -0.9924203218\n', ' H         -1.5961543407        2.0625756449        0.4226651515\n', ' H         -0.1240559598        0.2864369042        1.3610377216\n', ' H         -1.0526015016       -0.3133606987       -1.5017489184\n', ' H         -1.7197802379       -1.5206436563        1.2260423918\n', ' H         -2.3913236835       -2.0266289021       -0.3981073554\n', ' H          1.9706804557   

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

